In [1]:
#Carga de Librerias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os

In [2]:
Purchases_2016 = pd.read_csv("PurchasesFINAL12312016.csv")

In [5]:
Purchases_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372474 entries, 0 to 2372473
Data columns (total 16 columns):
 #   Column          Dtype         
---  ------          -----         
 0   InventoryId     object        
 1   Store           int64         
 2   Brand           int64         
 3   Description     object        
 4   Size            object        
 5   VendorNumber    int64         
 6   VendorName      object        
 7   PONumber        int64         
 8   PODate          datetime64[ns]
 9   ReceivingDate   datetime64[ns]
 10  InvoiceDate     datetime64[ns]
 11  PayDate         datetime64[ns]
 12  PurchasePrice   float64       
 13  Quantity        int64         
 14  Dollars         float64       
 15  Classification  int64         
dtypes: datetime64[ns](4), float64(2), int64(6), object(4)
memory usage: 289.6+ MB


In [4]:
#Convertir PODate, RecevingDate, InvoiceDate, PayDate en formato fecha

Purchases_2016['PODate'] = pd.to_datetime(Purchases_2016['PODate'])
Purchases_2016['ReceivingDate'] = pd.to_datetime(Purchases_2016['ReceivingDate'])
Purchases_2016['InvoiceDate'] = pd.to_datetime(Purchases_2016['InvoiceDate'])
Purchases_2016['PayDate'] = pd.to_datetime(Purchases_2016['PayDate'])


In [12]:
#Estadisticas de cada columna
for i in Purchases_2016.columns:
    print(i)
    #Esto imprime las estadísticas descriptivas (como count, mean, std, min, 25%, 50%, 75%, max) para esa columna.
    print(Purchases_2016[i].describe())
    print('--------------------------------------------------------------')

InventoryId
count               2324824
unique               241510
top       73_DONCASTER_8068
freq                    180
Name: InventoryId, dtype: object
--------------------------------------------------------------
Store
count    2.324824e+06
mean     4.471684e+01
std      2.348514e+01
min      1.000000e+00
25%      2.500000e+01
50%      4.800000e+01
75%      6.700000e+01
max      8.100000e+01
Name: Store, dtype: float64
--------------------------------------------------------------
Brand
count    2.324824e+06
mean     1.263551e+04
std      1.257689e+04
min      5.800000e+01
25%      3.696000e+03
50%      6.579000e+03
75%      1.927000e+04
max      9.063100e+04
Name: Brand, dtype: float64
--------------------------------------------------------------
Description
count                   2324824
unique                     9425
top       Tito's Handmade Vodka
freq                      14081
Name: Description, dtype: object
-------------------------------------------------------------

In [7]:
import re

def convertir_a_ml(size):
    # Expresión regular para extraer el número principal, unidad y posibles cantidades de paquetes
    match = re.match(r"(\d+(\.\d+)?)\s*(mL|L|liter|oz)?(?:\s*(\d+)?\s*Pk)?", str(size), re.IGNORECASE)
    
    if match:
        valor = float(match.group(1))  # Valor numérico principal
        unidad = match.group(3).lower() if match.group(3) else ''  # Unidad de medida
        cantidad_pk = int(match.group(4)) if match.group(4) else 1  # Cantidad en el paquete, por defecto 1
        
        # Convertir a mililitros según la unidad
        if unidad in ['l', 'liter']:
            return valor * 1000 * cantidad_pk  # Convertir litros a mililitros y multiplicar por cantidad
        elif unidad == 'oz':
            return valor * 29.5735 * cantidad_pk  # Convertir onzas a mililitros y multiplicar por cantidad
        elif unidad == 'ml':
            return valor * cantidad_pk  # Ya está en mililitros, multiplicar por cantidad
    return None

# Aplicar la función a la columna 'Size' para crear la columna 'Size_ml' en mL
Purchases_2016['Size_ml'] = Purchases_2016['Size'].apply(convertir_a_ml)

# Verificar el resultado
print(Purchases_2016[['Size', 'Size_ml']].head(10))

    Size  Size_ml
0  750mL    750.0
1  1.75L   1750.0
2  1.75L   1750.0
3  1.75L   1750.0
4  750mL    750.0
5  1.75L   1750.0
6  750mL    750.0
7  200mL    200.0
8  750mL    750.0
9  1.75L   1750.0


In [8]:
print(Purchases_2016['Size_ml'].unique())

[  750.      1750.       200.      1500.        50.            nan
   375.       100.      3000.       748.      1000.       500.
   187.       400.      5000.      4000.       561.       250.
   180.       800.       147.8675 18000.       300.       720.
   150.       330.      2250.       600.      1125.       650.617
  6000.     20000.       374.     19500.      4500.      9000.    ]


In [9]:
# Eliminar filas donde 'Size_ml' tiene valores NaN
Purchases_2016 = Purchases_2016.dropna(subset=['Size_ml'])

# Verificar el resultado
print(Purchases_2016['Size_ml'].unique())

[  750.      1750.       200.      1500.        50.       375.
   100.      3000.       748.      1000.       500.       187.
   400.      5000.      4000.       561.       250.       180.
   800.       147.8675 18000.       300.       720.       150.
   330.      2250.       600.      1125.       650.617   6000.
 20000.       374.     19500.      4500.      9000.    ]


In [11]:
del Purchases_2016['Size']

In [14]:
Purchases_2016.to_csv("Purchases_final_2016.csv")